# Pneumonia Detection: Transfer Learning with ResNet50
## Thesis Section: Improved Model
This notebook uses transfer learning with ResNet50, pre-trained on ImageNet, to detect pneumonia from chest X-ray images using the Kaggle Chest X-Ray Pneumonia dataset ([Kaggle link](https://www.kaggle.com/datasets/paultimothymooney/chest-xray-pneumonia)). By leveraging pre-trained features and fine-tuning, this approach aims to achieve higher accuracy (expected 90-95%) than the CNN from scratch, making it suitable for the web service backend.

The dataset is the same as in the CNN notebook, with class imbalance addressed via class weights. We use more aggressive data augmentation and fine-tune the last 50 layers of ResNet50 to adapt to the specific dataset.

In [ ]:
# Install Kaggle API to download dataset
!pip install -q kaggle

# Upload kaggle.json file
from google.colab import files
files.upload()  # Upload kaggle.json

# Set up Kaggle directory and permissions
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

# Download and unzip the chest X-ray pneumonia dataset
!kaggle datasets download -d paultimothymooney/chest-xray-pneumonia
!unzip -q chest-xray-pneumonia.zip -d chest_xray

## Data Exploration
We verify the dataset structure, noting the imbalance in the training set (1,341 normal vs. 3,875 pneumonia images).

In [ ]:
import os

# Define dataset paths
base_dir = 'chest_xray/chest_xray'
train_dir = os.path.join(base_dir, 'train')
val_dir = os.path.join(base_dir, 'val')
test_dir = os.path.join(base_dir, 'test')

# Count images
train_normal = len(os.listdir(os.path.join(train_dir, 'NORMAL')))
train_pneumonia = len(os.listdir(os.path.join(train_dir, 'PNEUMONIA')))
print(f'Training set: {train_normal} normal, {train_pneumonia} pneumonia images')
print(f'Validation set: {len(os.listdir(os.path.join(val_dir, "NORMAL")))} normal, {len(os.listdir(os.path.join(val_dir, "PNEUMONIA")))} pneumonia images')
print(f'Test set: {len(os.listdir(os.path.join(test_dir, "NORMAL")))} normal, {len(os.listdir(os.path.join(test_dir, "PNEUMONIA")))} pneumonia images')

## Data Preprocessing
Images are resized to 224x224 (standard for ResNet50) and augmented with rotation, zoom, shifts, and brightness adjustments to improve robustness.

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Parameters
img_height, img_width = 224, 224
batch_size = 32

# Data generators with augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    zoom_range=0.2,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    brightness_range=[0.8, 1.2]
)
val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

# Load data
train_data = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary'
)
val_data = val_datagen.flow_from_directory(
    val_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary'
)
test_data = test_datagen.flow_from_directory(
    test_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary',
    shuffle=False
)

## Class Weights
Class weights are computed to balance the training process, giving higher weight to the minority class.

In [ ]:
from sklearn.utils import class_weight
import numpy as np

# Compute class weights
labels = train_data.classes
weights = class_weight.compute_class_weight('balanced', classes=np.unique(labels), y=labels)
class_weights = dict(enumerate(weights))
print('Class weights:', class_weights)

## Model Architecture
We use ResNet50 pre-trained on ImageNet, freezing its layers initially and adding a custom classification head with GlobalAveragePooling2D, a dense layer, dropout, and a sigmoid output.

In [ ]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.optimizers import Adam

# Load ResNet50
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(img_height, img_width, 3))
for layer in base_model.layers:
    layer.trainable = False

# Add classification head
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)
output = Dense(1, activation='sigmoid')(x)
model = Model(inputs=base_model.input, outputs=output)

# Compile model
model.compile(optimizer=Adam(1e-4), loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

## Initial Training
We train the classification head for 10 epochs, keeping the base model frozen, using early stopping and model checkpointing to save the best model.

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# Callbacks
callbacks = [
    EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True),
    ModelCheckpoint('best_model.h5', monitor='val_loss', save_best_only=True)
]

# Train classification head
epochs_initial = 10
history = model.fit(
    train_data,
    epochs=epochs_initial,
    validation_data=val_data,
    class_weight=class_weights,
    callbacks=callbacks
)

## Fine-Tuning
We unfreeze the last 50 layers of ResNet50 and fine-tune with a lower learning rate (1e-5) for 10 epochs to adapt the model to the dataset.

In [ ]:
# Fine-tune last 50 layers
for layer in base_model.layers[-50:]:
    layer.trainable = True

# Recompile with lower learning rate
model.compile(optimizer=Adam(1e-5), loss='binary_crossentropy', metrics=['accuracy'])

# Continue training
epochs_finetune = 10
history_finetune = model.fit(
    train_data,
    epochs=epochs_finetune,
    validation_data=val_data,
    class_weight=class_weights,
    callbacks=callbacks
)

## Evaluation and Visualization
We evaluate the fine-tuned model on the test set and visualize combined training history and a confusion matrix.

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report

# Plot combined training history
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'] + history_finetune.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'] + history_finetune.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'] + history_finetune.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'] + history_finetune.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

# Evaluate on test set
test_loss, test_acc = model.evaluate(test_data)
print(f'Test Accuracy: {test_acc:.3f}')

# Generate confusion matrix
y_pred = (model.predict(test_data) > 0.5).astype(int)
y_true = test_data.classes
cm = confusion_matrix(y_true, y_pred)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()

# Classification report
print(classification_report(y_true, y_pred, target_names=['Normal', 'Pneumonia']))

## Model Saving
The fine-tuned model is saved for backend integration and downloaded.

In [ ]:
# Save and download model
model.save('resnet50_pneumonia.h5')
files.download('resnet50_pneumonia.h5')

## Discussion
The ResNet50 model, with transfer learning and fine-tuning, achieves higher accuracy (expected 90-95%) than the CNN from scratch, due to its pre-trained features and adaptation to the dataset. This model is suitable for the web service backend. The small validation set remains a limitation, but the test set performance confirms improved reliability.